### Web Scraping Realoka.com
#### Daffa Kaisha Pratama Chandra - 55770
---
##### Import library yang dibutuhkan & Persiapan BeautifulSoup4

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
baseurl = "https://www.realoka.com/" #base url

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}

---
### Rumah di Kabupaten Tangerang

In [3]:
url_tgr = []
t={}
c=0

for x in range(1,2): # page 1 sampai n
    k = requests.get('https://www.realoka.com/search.php?kw=&jid=1-Rumah&jsid=2&propinsi_id=36-Banten&kab_id=3603-Tangerang+Kab.&kt=&park=&smin=&smax=&minharga=&maxharga=&minhargasw=&maxhargasw=&furnish=&hrgper=&hadap=&akses='.format(x)).text #link page yang mau di scrape
    soup=BeautifulSoup(k,'html.parser')
    list_rumah = soup.find_all("div",{"class":"lisbox-hol clearfix"}) # cari card tempat produknya

    for rumah in list_rumah: # cari link produk
        link = rumah.find("a", href=True)
        if link:
            url_tgr.append(baseurl + link['href'])

In [4]:
# Banyak data
print("Banyak data rumah di Kabupaten Tangerang: {} rumah".format(len(url_tgr)))

Banyak data rumah di Kabupaten Tangerang: 36 rumah


In [5]:
data_tgr = [] # List untuk menampung data rumah yang telah di scraping
c = 0

for link in url_tgr:  # input data
    f = requests.get(link, headers=headers).text
    hun = BeautifulSoup(f, 'html.parser')

    # Harga
    try:
        price = hun.find("span", {"class": "detailpage-angka"}).text.replace('\n', "")
    except:
        price = None

    # Alamat
    try:
        location = hun.find("span", {"class": "post-address2"}).text.replace('\n', "")
    except:
        location = None

    # Deskripsi
    try:
        description = hun.find("div", {"class": "p-box-content padding-20 tx-rata iklan_desc word-wrap"}).text.replace('\n', "")
    except:
        description = None

    # Nama rumah
    try:
        name = hun.find("div", {"class": "post-title"}).text.replace('\n', "")
    except:
        name = None

    # Spesifikasi rumah 1
    try:
        spec_list = hun.find("ul", {"class": "rel-ls-none p-addtional-box tx-kiri"})
    except:
        spec_list = None

    spec_dict = {}

    # Ekstraksi spesifikasi rumah
    if spec_list:
        for spec in spec_list.find_all('li'):
            spec_name = spec.find('span', {'class': 'p-f-box-head'}).text
            spec_value = spec.find('span', {'class': 'p-f-box-info'}).text
            # Hapus kurung siku pada spec_value
            spec_value = spec_value.replace('[', '').replace(']', '')
            spec_dict[spec_name] = spec_value

    # Spesifikasi rumah 2
    try:
        spec_list2 = hun.find_all('li', {'class': 'col-md-4 col-sm-6b'})
    except:
        spec_list2 = None

    spec_dict2 = {}

    # Ekstraksi spesifikasi rumah 2
    if spec_list2:
        for specs in spec_list2:
            spec_values = specs.get_text().strip()
            spec_dict2[spec_values] = 1  # Set value = 1 jika terdapat spesifikasi yang tertera

    rmh_tgr = {
        "nama_rumah": name,
        "alamat": location,
        "harga": price,
        "deskripsi": description
    }

    rmh_tgr.update(spec_dict)
    rmh_tgr.update(spec_dict2)

    # Masukan data rumah ke list data_tgr
    data_tgr.append(rmh_tgr)
    c += 1
    print("completed", c)


completed 1
completed 2
completed 3
completed 4
completed 5
completed 6
completed 7
completed 8
completed 9
completed 10
completed 11
completed 12
completed 13
completed 14
completed 15
completed 16
completed 17
completed 18
completed 19
completed 20
completed 21
completed 22
completed 23
completed 24
completed 25
completed 26
completed 27
completed 28
completed 29
completed 30
completed 31
completed 32
completed 33
completed 34
completed 35
completed 36


In [6]:
# Convert spesifikasi rumah1 ke dataframe
df_spec = pd.DataFrame([spec_dict])
df_spec

,Tayang Sejak,Luas Tanah,Luas Bangunan,Kamar Tidur,Kamar Mandi,Sertifikat,Kondisi,Kelengkapan,Jumlah Lantai,Menghadap,Daya Listrik,Parkir
0,3 jam yang lalu,136 m2,200 m2,4,3,SHM - Sertifikat Hak Milik,Bekas,Kosongan,2,Timur,3300 VA,1 mobil


In [7]:
# Convert spesifikasi rumah2 ke dataframe
df_specs = pd.DataFrame([spec_dict2])
df_specs

,K. Pembantu,Air PAM,Balkon,One Gate,Pengamanan
0,1,1,1,1,1


In [8]:
# Gabungkan spesifikasi rumah 1 dengan spesifikasi rumah 2
specs = pd.concat([df_spec, df_specs], axis=1)
specs

,Tayang Sejak,Luas Tanah,Luas Bangunan,Kamar Tidur,Kamar Mandi,Sertifikat,Kondisi,Kelengkapan,Jumlah Lantai,Menghadap,Daya Listrik,Parkir,K. Pembantu,Air PAM,Balkon,One Gate,Pengamanan
0,3 jam yang lalu,136 m2,200 m2,4,3,SHM - Sertifikat Hak Milik,Bekas,Kosongan,2,Timur,3300 VA,1 mobil,1,1,1,1,1


In [9]:
# Covert data rumah ke dataframe
df_tgr = pd.DataFrame(data_tgr)

# Gabungkan data rumah dengan data spesifikasi rumah
df_kab = pd.concat([df_tgr, specs], axis=1)
df_kab.head()

,nama_rumah,alamat,harga,deskripsi,Tayang Sejak,Luas Tanah,Luas Bangunan,Kamar Tidur,Kamar Mandi,Sertifikat,...,Kelengkapan,Jumlah Lantai,Menghadap,Daya Listrik,Parkir,K. Pembantu,Air PAM,Balkon,One Gate,Pengamanan
0,Delrey Biztown Business TownHouse di BSD City,"Ipeka Plus Bsd, Lengkong Kulon, Kec. Pagedang...",4.174.000.000,Deskripsi dan SpesifikasiDelrey Biztown Busine...,1 hari yang lalu,96 m2,190 m2,3,3,"Lainnya (PPJB, Girik, Adat, dll.)",...,Kosongan,2,Timur,3300 VA,1 mobil,1.0,1.0,1.0,1.0,1.0
1,"New Cluster Puri Jaya, Cluster Maleo, dekat Ko...","Puri Jaya Pasar Kemis Nur Sodik, Tangerang Kab",440.000.000,Deskripsi dan SpesifikasiPuri Jaya adalah PERU...,4 hari yang lalu,60 m2,31 m2,2,1,SHM - Sertifikat Hak Milik,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cluster Tanamera Residence dekat Bandara Soett...,"Jalan Gatot Subroto Tanah Merah Sepatan, Tang...",297.000.000,Deskripsi dan SpesifikasiCLUSTER TANAMERA RESI...,5 hari yang lalu,60 m2,30 m2,2,1,SHM - Sertifikat Hak Milik,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Rumah Murah dekat Puspem Bayar 2juta Akad Ruma...,"Jalan Raya Puspem Tigaraksa Matagara, Tangera...",320.400.000,Deskripsi dan SpesifikasiCLUSTER RUMAHKU MATAG...,5 hari yang lalu,60 m2,30 m2,2,1,SHM - Sertifikat Hak Milik,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Cluster Terbaru Puri, Cluster Maleo, Banyak Pr...","Puri Jaya Pasar Kemis, Tangerang Kab",500.000.000,Deskripsi dan SpesifikasiPuri Jaya Pasar Kemis...,2 minggu yang lalu,60 m2,46 m2,2,2,SHM - Sertifikat Hak Milik,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Tambahkan variabel url dan wilayah
df_kab['Wilayah'] = 'Kabupaten Tangerang'
df_kab['url'] = url_tgr

In [11]:
df_kab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 46 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   nama_rumah     36 non-null     object 
 1   alamat         36 non-null     object 
 2   harga          36 non-null     object 
 3   deskripsi      36 non-null     object 
 4   Tayang Sejak   36 non-null     object 
 5   Luas Tanah     36 non-null     object 
 6   Luas Bangunan  36 non-null     object 
 7   Kamar Tidur    36 non-null     object 
 8   Kamar Mandi    36 non-null     object 
 9   Sertifikat     36 non-null     object 
 10  Kondisi        36 non-null     object 
 11  Kelengkapan    36 non-null     object 
 12  Jumlah Lantai  36 non-null     object 
 13  Menghadap      36 non-null     object 
 14  Daya Listrik   36 non-null     object 
 15  Parkir         36 non-null     object 
 16  One Gate       28 non-null     float64
 17  Pengamanan     34 non-null     float64
 18  Kolam Renang